# Segmenting and Clustering Neighborhoods in Toronto.ipynb

#### 1. Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#### Scraping the wikipedia site 

In [141]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
# headers
headers = []
for head in soup.find('table', class_='wikitable').find_all('th')[0::1]:
    headers.append(head.get_text()[:-1])


right_table=soup.find('table', class_='wikitable sortable')

postals = []
borough = []
neignbourhood = []

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postals.append(cells[0].find(text=True)[:-1])
        borough.append(cells[1].find(text=True)[:-1])
        neignbourhood.append(cells[2].find(text=True)[:-1])


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### 2. The Dataset

In [145]:
# The Dataframe
df = pd.DataFrame(postals,columns=['Postal Code'])
df['Borough'] = borough
df['Neighbourhood'] = neignbourhood
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### 3. cleaning the dataset(Ignoring cells with a borough that is Not assigned.)

In [148]:
data = df[df['Borough'] != 'Not assigned']
data.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### 4. The Shape of the dataset

In [149]:
df.shape

(180, 3)

#### 5. Getting the logitude and latitude through geocoder

In [151]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('MG5'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
(latitude, longitude)

NameError: name 'postal_code' is not defined